In [1]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
from collections import defaultdict
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
# import pickle

/tmp/ipykernel_3383997/2563711703.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# embeds_file = '/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES.pkl'
# with open(embeds_file, 'rb') as file:
#     # Load the data from the pickle file
#     data = pickle.load(file)

In [3]:
def code_data_type(df, meta, percent_missing):
    for i, col in enumerate(df.columns):
        if col == 'ID' or col == 'LONIUID.T1' or col == 'LONIUID.FLAIR':
            continue
        # print(col)
        # print(meta1[meta1['name'] == col]['type'].iloc[0])
        meta[col] = {}
        meta[col]['name'] = col
        meta[col]['unique_values'] = list(df[col].dropna().unique())
        meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
        meta[col]['length'] = 1
        mydtype = df[col].dtype
        if 'category' in str(mydtype):
            meta[col]['type'] = 'C'
        else:
            meta[col]['type'] = 'N'
        if col == 'img_MRI_T1':
            meta[col]['type'] = 'M'
        meta[col]['missing_rate'] = round(percent_missing[col], 2)
    df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]
    return df

# Load Data

In [5]:
# data = pd.read_csv('/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES.csv')
# data = pd.read_csv('/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES_single.csv')
data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/oasis_data_single.csv')

In [6]:
demog = data.columns[data.columns.str.contains('his')].to_list()
diagnosis = ['cd_DIAGNOSIS']
mmse = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MMSE')].to_list()
cdr = data.columns[data.columns.str.contains('cdr')].to_list()
cvd = data.columns[data.columns.str.contains('cvd')].to_list()
physical = data.columns[data.columns.str.contains('ph')].to_list()
moca = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MOCA')].to_list()
np = data.columns[(data.columns.str.contains('bat')) & (~data.columns.str.contains('MOCA')) & (~data.columns.str.contains('MMSE'))].to_list()
gds = data.columns[data.columns.str.contains('gds')].to_list()
npiq = data.columns[data.columns.str.contains('npiq')].to_list()
faq = data.columns[data.columns.str.contains('faq')].to_list()
hx = data.columns[data.columns.str.contains('iha')].to_list()
labels = ['amy_label', 'tau_label']
volumes = data.columns[data.columns.str.contains('FS')].to_list()

# ADNI

In [31]:
all_columns = demog + diagnosis + mmse + volumes + cdr + cvd + physical + moca + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + ['his_NACCAGE', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'gds_NACCGDS', 'npiq_NPISCORE', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [32]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
percent_missing

{'amy_label': 0.0,
 'tau_label': 0.4838300570703868,
 'ID': 0.0,
 'his_SEX': 0.0,
 'his_HISPANIC': 0.003804692454026633,
 'his_PRIMLANG': 0.0,
 'his_EDUC': 0.0,
 'his_MARISTAT': 0.0031705770450221942,
 'his_RESIDENC': 0.01585288522511097,
 'his_HANDED': 0.0006341154090044388,
 'his_NACCNIHR': 0.0057070386810399495,
 'his_NACCAGE': 0.0,
 'his_NACCMOM': 0.6119213696892835,
 'his_NACCDAD': 0.6169942929613189,
 'ph_HEIGHT': 0.8795180722891566,
 'ph_WEIGHT': 0.0063411540900443885,
 'ph_BPSYS': 0.007609384908053266,
 'ph_BPDIAS': 0.007609384908053266,
 'ph_HRATE': 0.010145846544071021,
 'cvd_ABRUPT': 0.18706404565630944,
 'cvd_STEPWISE': 0.18706404565630944,
 'cvd_SOMATIC': 0.186429930247305,
 'cvd_EMOT': 0.18706404565630944,
 'cvd_HXHYPER': 0.186429930247305,
 'cvd_HXSTROKE': 0.186429930247305,
 'cvd_FOCLSYM': 0.186429930247305,
 'cvd_FOCLSIGN': 0.186429930247305,
 'cvd_HACHIN': 0.186429930247305,
 'npiq_DEL': 0.5199746353836399,
 'npiq_HALL': 0.5199746353836399,
 'npiq_AGIT': 0.52060875079

In [ ]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

In [33]:
df = code_data_type(data, meta, percent_missing)
df.to_csv('../meta_files/meta_file.csv', index=False)

,name,missing_rate,type,num_unique_values,length,unique_values
amy_label,amy_label,0.0,C,2,1,"[0, 1]"
tau_label,tau_label,0.48,C,2,1,"[0.0, 1.0]"
his_SEX,his_SEX,0.0,C,2,1,"[female, male]"
his_HISPANIC,his_HISPANIC,0.0,C,2,1,"[no, yes]"
his_PRIMLANG,his_PRIMLANG,0.0,C,3,1,"[0.0, 2.0, 1.0]"
...,...,...,...,...,...,...
iha_Neurologic,iha_Neurologic,0.39,C,3,1,"[1.0, 0.0, 2.0]"
iha_Cardiovascular,iha_Cardiovascular,0.39,C,3,1,"[1.0, 0.0, 2.0]"
iha_Endocrine_Metabolic,iha_Endocrine_Metabolic,0.39,C,3,1,"[1.0, 0.0, 2.0]"
iha_Musculoskeletal,iha_Musculoskeletal,0.39,C,3,1,"[1.0, 0.0, 2.0]"


# OASIS

In [7]:
# don't need this, should have removed sooner
data.drop(columns=['bat_CRAFTDTI'], inplace = True)
np.remove('bat_CRAFTDTI')

In [8]:
all_columns = demog + diagnosis + mmse + volumes + cdr + cvd + physical + moca + gds + npiq + faq + hx + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + np + ['his_NACCAGE', 'his_SMOKYRS', 'his_QUITSMOK', 'his_PACKSPER', 'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCARECN', 'bat_NACCMOCA', 'gds_NACCGDS', 'faq_FAQTOTAL', 'cvd_HACHIN'] 
# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [9]:
data

,tau_label,amy_label,ID,his_SEX,his_HISPANIC,his_EDUC,his_MARISTAT,his_LIVSIT,his_INDEPEND,his_RESIDENC,his_HANDED,his_NACCAGE,his_NACCNIHR,his_NACCMOM,his_NACCDAD,his_TOBAC30,his_TOBAC100,his_SMOKYRS,his_PACKSPER,his_QUITSMOK,his_CVHATT,his_CVAFIB,his_CVANGIO,his_CVBYPASS,his_CVPACE,his_CVCHF,his_CVOTHR,his_CBSTROKE,his_CBTIA,his_PD,his_PDOTHR,his_SEIZURES,his_NCOTHR,his_DIABETES,his_HYPERTEN,his_HYPERCHO,his_B12DEF,his_THYROID,his_INCONTU,his_INCONTF,his_ALCOHOL,his_ABUSOTHR,his_PSYCDIS,ph_HEIGHT,ph_WEIGHT,ph_BPSYS,ph_BPDIAS,ph_HRATE,ph_VISION,ph_VISCORR,ph_VISWCORR,ph_HEARING,ph_HEARAID,ph_HEARWAID,cvd_ABRUPT,cvd_STEPWISE,cvd_SOMATIC,cvd_EMOT,cvd_HXHYPER,cvd_HXSTROKE,cvd_FOCLSYM,cvd_FOCLSIGN,cvd_HACHIN,cvd_CVDCOG,cvd_STROKCOG,cvd_CVDIMAG,npiq_DEL,npiq_HALL,npiq_AGIT,npiq_DEPD,npiq_ANX,npiq_ELAT,npiq_APA,npiq_DISN,npiq_IRR,npiq_MOT,npiq_NITE,npiq_APP,gds_SATIS,gds_DROPACT,gds_EMPTY,gds_BORED,gds_SPIRITS,gds_AFRAID,gds_HAPPY,gds_HELPLESS,gds_STAYHOME,gds_MEMPROB,gds_WONDRFUL,gds_WRTHLESS,gds_ENERGY,gds_HOPELESS,gds_BETTER,faq_BILLS,faq_TAXES,faq_SHOPPING,faq_GAMES,faq_STOVE,faq_MEALPREP,faq_EVENTS,faq_PAYATTN,faq_REMDATES,faq_TRAVEL,bat_NACCMMSE,bat_LOGIMEM,bat_MEMUNITS,bat_UDSBENTC,bat_UDSBENTD,bat_UDSBENRS,bat_ANIMALS,bat_VEG,bat_TRAILA,bat_TRAILB,bat_WAIS,bat_UDSVERFC,bat_UDSVERFN,bat_UDSVERNF,bat_UDSVERLC,bat_UDSVERLR,bat_UDSVERLN,bat_UDSVERTN,bat_UDSVERTE,bat_UDSVERTI,bat_MOCATRAI,bat_MOCACUBE,bat_MOCACLOC,bat_MOCACLON,bat_MOCACLOH,bat_MOCANAMI,bat_MOCADIGI,bat_MOCALETT,bat_MOCASER7,bat_MOCAREPE,bat_MOCAFLUE,bat_MOCAABST,bat_MOCARECN,bat_MOCAORDT,bat_MOCAORMO,bat_MOCAORYR,bat_MOCAORDY,bat_MOCAORPL,bat_MOCAORCT,bat_NACCMOCA,bat_CRAFTVRS,bat_CRAFTURS,bat_DIGFORCT,bat_DIGFORSL,bat_DIGBACCT,bat_DIGBACLS,bat_CRAFTDVR,bat_CRAFTDRE,bat_CRAFTCUE,bat_MINTTOTS,bat_MINTTOTW,bat_MINTSCNG,bat_MINTSCNC,bat_MINTPCNG,bat_MINTPCNC,cd_DIAGNOSIS,cdr_MEMORY,cdr_ORIENT,cdr_JUDGMENT,cdr_COMMUN,cdr_HOMEHOBB,cdr_PERSCARE,cdr_CDRSUM,cdr_CDRGLOB,FS_MTL_VOLUME,FS_NEO-T_VOLUME,FS_TEMPO_PARIETAL_VOLUME,FS_FRONTAL_VOLUME,gds_NACCGDS,faq_FAQTOTAL
0,0.0,0.0,OAS30003,female,no,18.0,0.0,1.0,0.0,0.0,1.0,69.028621,whi,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,160.000000,118.0,78.0,70.0,NaN,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,NaN,NaN,NaN,NaN,NaN,17.0,NaN,25.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625.600,9435.50,7302.576923,10576.12,0.0,0.0
1,0.0,0.0,OAS30004,female,no,17.0,0.0,1.0,0.0,0.0,0.0,64.598950,whi,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,202.500000,147.0,89.0,76.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,26.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1413.725,9448.50,7611.230769,10584.10,1.0,0.0
2,0.0,0.0,OAS30005,female,NaN,16.0,4.0,2.0,0.0,0.0,1.0,57.516799,asi,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.5,178.000000,120.0,78.0,64.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,Na

In [10]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}

filtered_percent_missing

{'cvd_STROKCOG': 0.9779874213836478, 'cvd_CVDIMAG': 0.9727463312368972}

In [11]:
df = code_data_type(data, meta, percent_missing)
df

,name,missing_rate,type,num_unique_values,length,unique_values
tau_label,tau_label,0.61,C,2,1,"[0.0, 1.0]"
amy_label,amy_label,0.0,C,2,1,"[0.0, 1.0]"
his_SEX,his_SEX,0.0,C,2,1,"[female, male]"
his_HISPANIC,his_HISPANIC,0.07,C,2,1,"[no, yes]"
his_EDUC,his_EDUC,0.0,N,18,1,"[18.0, 17.0, 16.0, 20.0, 14.0, 12.0, 13.0, 19...."
...,...,...,...,...,...,...
FS_NEO-T_VOLUME,FS_NEO-T_VOLUME,0.0,N,926,1,"[9435.5, 9448.5, 11130.25, 7241.25, 8125.25, 8..."
FS_TEMPO_PARIETAL_VOLUME,FS_TEMPO_PARIETAL_VOLUME,0.0,N,949,1,"[7302.576923076923, 7611.2307692307695, 7909.8..."
FS_FRONTAL_VOLUME,FS_FRONTAL_VOLUME,0.0,N,952,1,"[10576.12, 10584.1, 10850.35, 9346.37, 9973.06..."
gds_NACCGDS,gds_NACCGDS,0.0,N,13,1,"[0.0, 1.0, 2.0, 3.0, 7.0, 5.0, 10.0, 8.0, 4.0,..."


In [12]:
print(all_columns)

['his_SEX', 'his_HISPANIC', 'his_EDUC', 'his_MARISTAT', 'his_LIVSIT', 'his_INDEPEND', 'his_RESIDENC', 'his_HANDED', 'his_NACCAGE', 'his_NACCNIHR', 'his_NACCMOM', 'his_NACCDAD', 'his_TOBAC30', 'his_TOBAC100', 'his_SMOKYRS', 'his_PACKSPER', 'his_QUITSMOK', 'his_CVHATT', 'his_CVAFIB', 'his_CVANGIO', 'his_CVBYPASS', 'his_CVPACE', 'his_CVCHF', 'his_CVOTHR', 'his_CBSTROKE', 'his_CBTIA', 'his_PD', 'his_PDOTHR', 'his_SEIZURES', 'his_NCOTHR', 'his_DIABETES', 'his_HYPERTEN', 'his_HYPERCHO', 'his_B12DEF', 'his_THYROID', 'his_INCONTU', 'his_INCONTF', 'his_ALCOHOL', 'his_ABUSOTHR', 'his_PSYCDIS', 'cd_DIAGNOSIS', 'bat_NACCMMSE', 'FS_MTL_VOLUME', 'FS_NEO-T_VOLUME', 'FS_TEMPO_PARIETAL_VOLUME', 'FS_FRONTAL_VOLUME', 'cdr_MEMORY', 'cdr_ORIENT', 'cdr_JUDGMENT', 'cdr_COMMUN', 'cdr_HOMEHOBB', 'cdr_PERSCARE', 'cdr_CDRSUM', 'cdr_CDRGLOB', 'cvd_ABRUPT', 'cvd_STEPWISE', 'cvd_SOMATIC', 'cvd_EMOT', 'cvd_HXHYPER', 'cvd_HXSTROKE', 'cvd_FOCLSYM', 'cvd_FOCLSIGN', 'cvd_HACHIN', 'cvd_CVDCOG', 'cvd_STROKCOG', 'cvd_CVDIM

In [13]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

tau_label
0.0    317
1.0     56
Name: count, dtype: int64
amy_label
0.0    609
1.0    345
Name: count, dtype: int64
his_SEX
female    533
male      421
Name: count, dtype: int64
his_HISPANIC
no     889
yes      1
Name: count, dtype: int64
his_MARISTAT
0.0    645
2.0    123
1.0    114
4.0     55
3.0     10
5.0      6
Name: count, dtype: int64
his_LIVSIT
1.0    635
0.0    231
2.0     59
4.0     25
3.0      4
Name: count, dtype: int64
his_INDEPEND
0.0    891
1.0     53
2.0      8
3.0      1
Name: count, dtype: int64
his_RESIDENC
0.0    907
1.0     43
2.0      1
Name: count, dtype: int64
his_HANDED
1.0    859
0.0     83
2.0      2
Name: count, dtype: int64
his_NACCNIHR
whi    813
blk    131
asi      6
mul      3
ind      1
Name: count, dtype: int64
his_NACCMOM
0.0    448
1.0    409
Name: count, dtype: int64
his_NACCDAD
0.0    615
1.0    212
Name: count, dtype: int64
his_TOBAC30
0.0    527
1.0     37
Name: count, dtype: int64
his_TOBAC100
0.0    307
1.0    251
Name: count, dtype: int64
his_

In [14]:
df.to_csv('../meta_files/oasis_meta_file.csv', index=False)